# Proyek: Analisis Prediksi Churn Telco
## Notebook 01: Data Preparation

**Fase CRISP-ML(Q):** 3. Data Preparation

**Tujuan:**
1.  Memuat data mentah dari `data/raw/`.
2.  Menerapkan perbaikan data (anomali `TotalCharges`) yang ditemukan di Notebook 00.
3.  Merancang dan memvalidasi `Pipeline` preprocessing (`ColumnTransformer`) untuk menangani fitur numerik dan kategorikal.
4.  (Opsional) Menyimpan *snapshot* data yang telah diproses ke `data/prepared/` **hanya untuk tujuan inspeksi manual**, bukan untuk pemodelan akhir.

## 1. Setup & Pemuatan Data

In [1]:
import pandas as pd
import numpy as np
import os

# Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Definisikan path kita
RAW_DATA_PATH = '../data/raw/telco_churn.csv'
PREPARED_DATA_DIR = '../data/prepared/'
PREPARED_DATA_PATH = os.path.join(PREPARED_DATA_DIR, 'telco_features_INSPECTION.csv')

print("Libraries and paths defined.")

Libraries and paths defined.


In [2]:
df = pd.read_csv(RAW_DATA_PATH)
print(f"Data mentah loaded. Shape: {df.shape}")

Data mentah loaded. Shape: (7043, 21)


## 2. Pembersihan Data (Menangani Anomali)

Seperti yang kita temukan di Notebook 00, kita harus memperbaiki `TotalCharges`.

In [3]:
# 1. Konversi 'TotalCharges' ke numerik. 'errors='coerce'' akan mengubah string " " menjadi NaN.
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 2. Buang baris NaN (11 baris nasabah 'tenure=0' yang kita temukan)
df_clean = df.dropna(subset=['TotalCharges']).copy()

print(f"Shape lama: {df.shape}")
print(f"Shape baru setelah dropna: {df_clean.shape}")
print("'TotalCharges' fixed and NaNs dropped.")

Shape lama: (7043, 21)
Shape baru setelah dropna: (7032, 21)
'TotalCharges' fixed and NaNs dropped.


## 3. Desain "Resep" Preprocessing (`ColumnTransformer`)

Kita akan merancang `preprocessor` yang akan menjadi inti dari *pipeline* pemodelan kita nanti. Kita perlu memisahkan fitur berdasarkan tipe datanya.

In [4]:
# 1. Tentukan Fitur (X) dan Target (y)
X = df_clean.drop(columns=['Churn', 'customerID']) # Kita buang target & ID
y = df_clean['Churn']

# 2. Identifikasi grup fitur
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']

# 'SeniorCitizen' sudah 0/1, jadi kita tidak perlu meng-encode-nya.
# Kita akan gunakan 'passthrough'.
passthrough_features = ['SeniorCitizen']

# Sisa fitur adalah kategorikal yang perlu di One-Hot Encode
categorical_features = [
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 
    'PaperlessBilling', 'PaymentMethod'
]

print(f"{len(numeric_features)} fitur numerik diidentifikasi.")
print(f"{len(categorical_features)} fitur kategorikal diidentifikasi.")

3 fitur numerik diidentifikasi.
15 fitur kategorikal diidentifikasi.


In [5]:
# 3. Buat transformer untuk setiap tipe

# Pipeline untuk fitur numerik: scaling
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline untuk fitur kategorikal: one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# 4. Gabungkan transformer menggunakan ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Ini akan membiarkan 'SeniorCitizen' lolos tanpa diubah
)

print("ColumnTransformer 'preprocessor' berhasil dirancang.")


ColumnTransformer 'preprocessor' berhasil dirancang.


## 4. Validasi "Resep"

Mari kita terapkan `preprocessor` yang baru kita rancang ke data `X` untuk memvalidasi bahwa outputnya terlihat benar dan siap untuk ML.

In [6]:
# Terapkan preprocessor (fit dan transform)
print("Menerapkan preprocessor ke data X...")
X_processed = preprocessor.fit_transform(X)

# Dapatkan nama fitur setelah OHE
try:
    feature_names = preprocessor.get_feature_names_out()
except Exception:
    # Fallback untuk sklearn versi lama (jika ada)
    ohe_features = preprocessor.named_transformers_['cat']\
                                  .named_steps['onehot']\
                                  .get_feature_names_out(categorical_features)
    feature_names = numeric_features + list(ohe_features) + passthrough_features

# Ubah kembali ke DataFrame untuk inspeksi
X_processed_df = pd.DataFrame(X_processed, columns=feature_names)

print(f"Data berhasil diproses. Shape baru: {X_processed_df.shape}")
print("\n--- Head of Processed Data ---")
print(X_processed_df.head())

Menerapkan preprocessor ke data X...
Data berhasil diproses. Shape baru: (7032, 30)

--- Head of Processed Data ---
   num__tenure  num__MonthlyCharges  num__TotalCharges  cat__gender_Male  \
0    -1.280248            -1.161694          -0.994194               0.0   
1     0.064303            -0.260878          -0.173740               1.0   
2    -1.239504            -0.363923          -0.959649               1.0   
3     0.512486            -0.747850          -0.195248               1.0   
4    -1.239504             0.196178          -0.940457               0.0   

   cat__Partner_Yes  cat__Dependents_Yes  cat__PhoneService_Yes  \
0               1.0                  0.0                    0.0   
1               0.0                  0.0                    1.0   
2               0.0                  0.0                    1.0   
3               0.0                  0.0                    0.0   
4               0.0                  0.0                    1.0   

   cat__MultipleLines_No

In [7]:
print("--- Statistik Data yang Diproses (Inspeksi) ---")
print(X_processed_df.describe().T)

--- Statistik Data yang Diproses (Inspeksi) ---
                                             count          mean       std  \
num__tenure                                 7032.0 -1.126643e-16  1.000071   
num__MonthlyCharges                         7032.0  6.062651e-17  1.000071   
num__TotalCharges                           7032.0 -1.119064e-16  1.000071   
cat__gender_Male                            7032.0  5.046928e-01  0.500014   
cat__Partner_Yes                            7032.0  4.825085e-01  0.499729   
cat__Dependents_Yes                         7032.0  2.984926e-01  0.457629   
cat__PhoneService_Yes                       7032.0  9.032992e-01  0.295571   
cat__MultipleLines_No phone service         7032.0  9.670080e-02  0.295571   
cat__MultipleLines_Yes                      7032.0  4.219283e-01  0.493902   
cat__InternetService_Fiber optic            7032.0  4.402730e-01  0.496455   
cat__InternetService_No                     7032.0  2.161547e-01  0.411650   
cat__OnlineSecur

**Temuan Validasi:**

Tabel `.describe()` di atas adalah bukti keberhasilan kita:
1.  Fitur numerik (seperti `num__tenure`, `num__MonthlyCharges`) sekarang memiliki `mean` (rata-rata) mendekati 0 dan `std` (standar deviasi) 1.0. **Scaling berhasil.**
2.  Fitur kategorikal (seperti `cat__Contract_OneYear`, `cat__InternetService_FiberOptic`) sekarang adalah kolom biner 0/1. **OHE berhasil.**
3.  Fitur `remainder__SeniorCitizen` (yang kita *passthrough*) masih dalam rentang 0-1. **Passthrough berhasil.**

"Resep" `preprocessor` kita sudah terbukti dan siap digunakan.

## 5. (Hanya Opsional) Simpan Snapshot Inspeksi

Kita akan menyimpan *snapshot* ini ke `data/prepared/` **hanya untuk tujuan inspeksi manual**. Kita tidak akan menggunakan file ini dalam pemodelan.

In [ ]:
# Pastikan direktori 'prepared' ada
os.makedirs(PREPARED_DATA_DIR, exist_ok=True)

# Kita gabungkan kembali target 'y' agar file inspeksi kita lengkap
df_final_inspection = X_processed_df.copy()
df_final_inspection['Churn'] = y.reset_index(drop=True) # Reset index y agar cocok

# Simpan ke CSV
try:
    df_final_inspection.to_csv(PREPARED_DATA_PATH, index=False)
    print(f"Data inspeksi berhasil disimpan ke: {PREPARED_DATA_PATH}")
except Exception as e:
    print(f"Error saat menyimpan file: {e}")

Data inspeksi berhasil disimpan ke: ../data/prepared/telco_features_INSPECTION.csv


## Kesimpulan Fase 3

Kita telah berhasil mengonversi data mentah `telco_churn.csv` (7043 baris, 21 kolom) menjadi data yang siap untuk ML (7032 baris, [misal] 27 kolom hasil OHE).

Yang paling penting, kita telah **merancang dan memvalidasi `preprocessor`** (objek `ColumnTransformer`) kita.

Kita siap untuk menyalin logika `preprocessor` ini ke **Langkah 02: Modeling Churn Classification**.